In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_selection import f_regression
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

data_df = None
if os.path.isfile("data/data.pkl"):
    data_df = pd.read_pickle("data/data.pkl")
else:
    print("Load Data")
data_df.head()

RANDOM_STATE = 42

In [ ]:
# Define X and Y
ad_hc_df = data_df[
    data_df["Diagnosis"].isin(["AD", "HC"]) & (data_df["Harmonized"].notna())
].copy()
tbi_df = data_df[
    data_df["Diagnosis"].isin(["NEG", "POS"]) & (data_df["Harmonized"].notna())
].copy()
mci_df = data_df[
    data_df["Diagnosis"].isin(["MCI", "HC"]) & (data_df["Harmonized"].notna())
].copy()

X_ad_hc = np.vstack(ad_hc_df["EVC"].values)
X_tbi = np.vstack(tbi_df["EVC"].values)
X_mci = np.vstack(mci_df["EVC"].values)

X_mci = StandardScaler().fit_transform(X_mci)
X_tbi = StandardScaler().fit_transform(X_tbi)
X_ad_hc = StandardScaler().fit_transform(X_ad_hc)

# X_mci = MinMaxScaler().fit_transform(X_mci)
# X_tbi = MinMaxScaler().fit_transform(X_tbi)
# X_ad_hc = MinMaxScaler().fit_transform(X_ad_hc)

y_ad_hc = ad_hc_df["Diagnosis"].map({"AD": 1, "HC": 0}).values
y_tbi = tbi_df["Diagnosis"].map({"POS": 1, "NEG": 0}).values
y_mci = mci_df["Diagnosis"].map({"MCI": 1, "HC": 0}).values

In [ ]:
def getCoefs(x, y):
    mdl = LogisticRegressionCV(
        solver="liblinear",
        random_state=RANDOM_STATE,
        class_weight="balanced",
        cv=LeaveOneOut(),
        n_jobs=-1,
    )
    mdl.fit(x, y)
    coefs = mdl.coef_
    coefs = -1 + 2 * ((coefs - coefs.min()) / (coefs.max() - coefs.min()))
    # coefs = MinMaxScaler(feature_range=(-1,1)).fit_transform(coefs)
    return coefs.flatten()


def calculate_r2(x, y):
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    y_pred = model.predict(x.reshape(-1, 1))
    return r2_score(y, y_pred)


coefs_ad = getCoefs(X_ad_hc, y_ad_hc)
coefs_tbi = getCoefs(X_tbi, y_tbi)
#coefs_mci = getCoefs(X_mci, y_mci)
#coefs_mci = []

In [ ]:
def highlightInformative(x, y, axis):
    residuals = np.abs(y - x)
    magnitudes = np.abs(x) + np.abs(y)
    weights = -1 * residuals + magnitudes
    top_10_indices = np.argsort(weights)[-10:][::-1]
    axis.scatter(x[top_10_indices], y[top_10_indices], alpha=0.9, c="red", marker="2")
    for idx in top_10_indices:
        print(f"{idx}\t{weights[idx]}")
    print("==========\n")

coefs_mci = coefs_ad

fig, axes = plt.subplots(1, 3, figsize=(10, 12))

r2_tbi_mci = calculate_r2(coefs_mci, coefs_tbi)
axes[0].scatter(coefs_mci, coefs_tbi, alpha=0.7, c="black", marker="2")
axes[0].plot([-1, 1], [-1, 1], linestyle="--", color="black", alpha=0.5)
axes[0].set_xlabel("Regression Coefficient (AD vs HC)")
axes[0].set_ylabel("Regression Coefficient (TBI+ vs TBI-)")
axes[0].set_title(f"TBI vs MCI (R^2 = {r2_tbi_mci:.2e})")
axes[0].grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
axes[0].set_xlim([-1, 1])
axes[0].set_ylim([-1, 1])
axes[0].set_aspect("equal", adjustable="box")
highlightInformative(coefs_mci, coefs_tbi, axes[0])

r2_ad_hc_tbi = calculate_r2(coefs_ad, coefs_tbi)
axes[1].scatter(coefs_ad, coefs_tbi, alpha=0.7, c="black", marker="2")
axes[1].plot([-1, 1], [-1, 1], linestyle="--", color="black", alpha=0.5)
axes[1].set_xlabel("Regression Coefficient (AD vs HC)")
axes[1].set_ylabel("Regression Coefficient (TBI+ vs TBI-)")
axes[1].set_title(f"TBI vs AD (R^2 = {r2_ad_hc_tbi:.2e})")
axes[1].grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
axes[1].set_xlim([-1, 1])
axes[1].set_ylim([-1, 1])
axes[1].set_aspect("equal", adjustable="box")
highlightInformative(coefs_ad, coefs_tbi, axes[1])

r2_ad_hc_mci = calculate_r2(coefs_ad, coefs_mci)
axes[2].scatter(coefs_ad, coefs_mci, alpha=0.7, c="black", marker="2")
axes[2].plot([-1, 1], [-1, 1], linestyle="--", color="black", alpha=0.5)
axes[2].set_xlabel("Regression Coefficient (AD vs HC)")
axes[2].set_ylabel("Regression Coefficient (MCI vs HC)")
axes[2].set_title(f"MCI vs AD (R^2 = {r2_ad_hc_mci:.2e})")
axes[2].grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
axes[2].set_xlim([-1, 1])
axes[2].set_ylim([-1, 1])
axes[2].set_aspect("equal", adjustable="box")
highlightInformative(coefs_ad, coefs_mci, axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Run statistical tests and multiple comparisons corrections
# p_values_ad_hc = np.array([ttest_ind(X_ad_hc[y_ad_hc == 1, i], X_ad_hc[y_ad_hc == 0, i], equal_var=False)[1]
#                             for i in range(X_ad_hc.shape[1])])

# p_values_tbi = np.array([ttest_ind(X_tbi[y_tbi == 1, i], X_tbi[y_tbi == 0, i], equal_var=False)[1]
#                           for i in range(X_tbi.shape[1])])
f_stat_ad, p_values_ad = f_regression(X_ad_hc, y_ad_hc)
f_stat_tbi, p_values_tbi = f_regression(X_tbi, y_tbi)
f_stat_mci, p_values_mci = f_regression(X_mci, y_mci)

_, p_values_ad_hc_corrected, _, _ = multipletests(p_values_ad, method="bonferroni")
_, p_values_tbi_corrected, _, _ = multipletests(p_values_tbi, method="bonferroni")
_, p_values_mci_corrected, _, _ = multipletests(p_values_mci, method="bonferroni")

significant_ad_hc = np.where(p_values_ad_hc_corrected < 0.05)[0]
significant_tbi = np.where(p_values_tbi_corrected < 0.05)[0]
significant_mci = np.where(p_values_mci_corrected < 0.05)[0]

print(f"P Values AD/HC: {p_values_ad}", end="\n========\n\n")
print(f"P Values TBI+/TBI-: {p_values_tbi}", end="\n========\n\n")
print(f"P Values MCI/HC: {p_values_mci}", end="\n========\n\n")